# Combining the NYC School Survey with the federal Civil Rights survey

This notebook documents how we combined the NYC School Survey from the NYC Deparment of Education with the federal Civil Rights Data Collection survey from the U.S. Department of Education for our analysis of bullying/harassment during the 2013-14 school year in NYC public schools.

## Import Python libraries and set working directories

In [2]:
import os
import feather
import numpy as np
import pandas as pd
import zipfile

In [13]:
input_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'input')
intermediate_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'intermediate')
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'output')
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')

The files in input.zip must first be unzipped to run this notebook.

In [16]:
with zipfile.ZipFile(os.path.join(data_dir, 'input.zip'), 'r') as zip_ref:
    zip_ref.extractall(data_dir)

## Run survey processing notebooks

Run the processing notebooks for both surveys:

In [3]:
%%capture
%run nyc_school_survey.ipynb
%run federal_civil_rights_survey.ipynb

## Create a crosswalk between the different school IDs, used at different levels of government: local NYC (`DBN`) to federal (`combokey`)

The `DBN` code is used to uniquely identify schools in the NYC School Survey, while the `combokey` used in the federal Civil Rights Data Collection. Since we're only looking to combine NYC schools, we'll start with the NYC School Survey `DBN` codes:

## Local (`DBN`)

The NYC Department of Education (NYCDOE) uses a **6-digit alphanumeric `DBN` (District Borough Number),** to identify schools. The code begins with the school's district number (2 digits), then the borough code (1 letter: K = Brooklyn; X = Bronx; Q = Queens; M = Manhattan; R = Staten Island), then the school code (3 digits).

Let's take a look at one of the intermediate files we created in `nyc_school_survey.ipynb` to see this in action:

In [4]:
df_nyc = pd.read_feather(
    os.path.join(intermediate_dir, 'nyc_survey_ids.feather')
)

df_nyc.head()

,DBN,School Name
0,01M015,P.S. 015 Roberto Clemente
1,01M019,P.S. 019 Asher Levy
2,01M020,P.S. 020 Anna Silver
3,01M034,P.S. 034 Franklin D. Roosevelt
4,01M063,The STAR Academy - P.S.63


## Local (`DBN`) to State (`BEDS`)

`DBN` IDs are unique to New York City's data collection system. In order to link them to the federal school IDs, we will first need to link them to the state school IDs.

The New York State Education Department (NYSED) uses a **12-digit `BEDS` (Basic Educational Data System) number** to identify schools.

The NYC Open Data Portal provides a 2013-2014 School Locations [file](https://data.cityofnewyork.us/Education/2013-2014-School-Locations/ac4n-c5re), which identifies schools using both the 12-digit BEDS number as well as a **6-digit `ATS` (Automate the Schools) code**. For the NYC schools in our dataset, the `ATS` is the same number as the `DBN`, so we can use this file to link the `DBN` to the `BEDS` for each school. For more information on the ATS and DBN systems, see NYCDOE's report, ["A collection of Acronyms and Jargon"](http://schools.nyc.gov/NR/rdonlyres/0C56D9B8-8DDB-46B8-AA5F-BF77AE8C2803/0/ACRONYMReferenceGuide.pdf) (last updated February 7, 2018). 

In [5]:
df_beds = pd.read_csv(
    os.path.join(input_dir, '2013_-_2014_School_Locations.csv'), 
    dtype = str
)

df_beds.columns = df_beds.columns.str.lower()

Remove some trailing spaces in the `ats system code` column and select relevant columns

In [6]:
df_beds['ats system code'] = df_beds['ats system code'].str.strip()
df_beds = df_beds[['ats system code', 'beds number', 'location_name', 'primary_address_line_1']]
df_beds.head()

,ats system code,beds number,location_name,primary_address_line_1
0,01M015,310100010015,P.S. 015 Roberto Clemente,333 EAST 4 STREET
1,01M019,310100010019,P.S. 019 Asher Levy,185 1 AVENUE
2,01M020,310100010020,P.S. 020 Anna Silver,166 ESSEX STREET
3,01M034,310100010034,P.S. 034 Franklin D. Roosevelt,730 EAST 12 STREET
4,01M063,310100010063,The STAR Academy - P.S.63,121 EAST 3 STREET


Merge this dataframe with the `df_nyc` dataframe in order to map `DBN` to `beds number`

In [7]:
df_nyc_beds = pd.merge(df_nyc, 
                          df_beds, 
                          left_on = 'DBN', 
                          right_on = 'ats system code', 
                          how = 'left', 
                          indicator = True)

Most of the unmatched schools include YABC (Young Adult Borough Centers), which are evening programs for high schoolers at-risk of dropping out.

In [8]:
df_nyc_beds._merge.value_counts()

both          1754
left_only       30
right_only       0
Name: _merge, dtype: int64

In [9]:
df_nyc_beds.loc[df_nyc_beds['_merge'] == 'left_only']['School Name'].values

array(['Washington Irving Yabc', 'George Washington Yabc',
       'Alfred E. Smith Campus YABC', 'Herbert H. Lehman Yabc',
       'Stevenson Yabc', 'Walton Yabc', 'Kennedy Yabc', 'Grace Dodge Yabc',
       'C Columbus Hs Yabc', 'Learning To Work Yabc At Monroe Academy',
       'Downtown Brooklyn Yabc', 'Automotive High School Yabc',
       "P.S. 372 -The Children's School", 'BOYS AND GIRLS YABC',
       'Erasmus Yabc', 'South Shore Educational Complex Yabc',
       'Thomas Jefferson Yabc', 'Franklin K. Lane Campus YABC',
       'Franklin Delano Roosevelt YABC', 'Abraham Lincoln Yabc',
       'HS Arts & Business Yabc', 'Flushing Yabc', 'John Adams Yabc',
       'Staten Island Yabc', 'Achievement First High School',
       'Uncommmon Charter HS 2', 'Achievement First University Prep',
       'Uncommon Charter HS', 'Harlem Village Academy HS',
       'KIPP NYC College Prep'], dtype=object)

These, along with a couple of charter schools (e.g., Uncommon Charter HS 2) and special education schools (e.g., P.S. 372 - The Children's School), will not be included in our final analysis.

In [10]:
df_nyc_beds = df_nyc_beds.loc[df_nyc_beds['_merge'] == 'both']
df_nyc_beds.drop(['_merge', 'ats system code'], axis = 1, inplace = True)

Now we have a dataset with both a `DBN` code and a `beds number` for each school.

In [11]:
df_nyc_beds.head()

,DBN,School Name,beds number,location_name,primary_address_line_1
0,01M015,P.S. 015 Roberto Clemente,310100010015,P.S. 015 Roberto Clemente,333 EAST 4 STREET
1,01M019,P.S. 019 Asher Levy,310100010019,P.S. 019 Asher Levy,185 1 AVENUE
2,01M020,P.S. 020 Anna Silver,310100010020,P.S. 020 Anna Silver,166 ESSEX STREET
3,01M034,P.S. 034 Franklin D. Roosevelt,310100010034,P.S. 034 Franklin D. Roosevelt,730 EAST 12 STREET
4,01M063,The STAR Academy - P.S.63,310100010063,The STAR Academy - P.S.63,121 EAST 3 STREET


## State (`BEDS`) to Federal (`NCESSCH`)

The National Center of Education Statistics (NCES), part of the U.S. Department of Education (ED), is the ["primary federal entity for collecting and analyzing data related to education in the U.S. and other nations"](https://nces.ed.gov/about/). NCES identifies schools using a 12-digit code known as the NCES id or `NCESSCH`. The school universe data [file](https://nces.ed.gov/ccd/Data/zip/EDGE_GEOIDS_201415_PUBLIC_SCHOOL_csv.zip) for the 2013-14 school year, along with files for previous years, can be found on the NCES's Common Core of Data [website](https://nces.ed.gov/ccd/pubschuniv.asp).

In [12]:
df_nces_raw = pd.read_csv(
    os.path.join(input_dir, 'sc132a.txt'), 
    delimiter='\t', 
    dtype = str
)
df_nces_raw.columns = df_nces_raw.columns.str.lower()

In [13]:
df_nces_raw.head()

,survyear,ncessch,fipst,leaid,schno,stid,seasch,leanm,schnam,phone,...,hpalm,hpalf,tr,tralm,tralf,toteth,virtualstat,nslpstatus,chartauth1,chartauth2
0,2013,010000200277,01,0100002,00277,210,0020,ALABAMA YOUTH SERVICES,SEQUOYAH SCH - CHALKVILLE CAMPUS,2056808574,...,-1,-1,-1,-1,-1,-1,VIRTUALNO,M,N,N
1,2013,010000201402,01,0100002,01402,210,0025,ALABAMA YOUTH SERVICES,EUFAULA SCH - EUFAULA CAMPUS,3346874441,...,-2,-2,-2,-2,-2,-2,VIRTUALNO,M,N,N
2,2013,010000201667,01,0100002,01667,210,0050,ALABAMA YOUTH SERVICES,CAMPS,3342153850,...,-1,-1,-1,-1,-1,-1,VIRTUALNO,M,N,N
3,2013,010000201670,01,0100002,01670,210,0060,ALABAMA YOUTH SERVICES,DET CTR,3342153850,...,-1,-1,-1,-1,-1,-1,VIRTUALNO,M,N,N
4,2013,010000201705,01,0100002,01705,210,0030,ALABAMA YOUTH SERVICES,WALLACE SCH - MT MEIGS CAMPUS,3342156039,...,-1,-1,-1,-1,-1,-1,VIRTUALNO,M,N,N


The federal NCES data has a column labeled **`seasch`, which is each state's own ID for each school.** For schools in New York City, this is the same value as the 12-digit BEDS code used by the New York State Department of Education. 

Let's merge the federal dataset with the schools from the NYC School Survey universe using the `seasch` and the `beds number`.

In [14]:
df_nces_raw = df_nces_raw[['ncessch', 'seasch', 'schnam', 'lcity', 'lstree']]
df_merge = pd.merge(df_nyc_beds, 
                    df_nces_raw, 
                    left_on = 'beds number', 
                    right_on = 'seasch', 
                    how = 'left', 
                    indicator = True)

Only one school, Icahn Charter School 3, seems to be off (looks like it's mixed up with Bronx Academy of Promise Charter School) - we can fix this manually.

In [15]:
df_merge._merge.value_counts()

both          1753
left_only        1
right_only       0
Name: _merge, dtype: int64

In [16]:
df_merge[df_merge['_merge'] == 'left_only']

,DBN,School Name,beds number,location_name,primary_address_line_1,ncessch,seasch,schnam,lcity,lstree,_merge
1734,84X422,Icahn Charter School 3,320900860917,Icahn Charter School 3,1500 PELHAM PARKWAY SOUTH,NaN,NaN,NaN,NaN,NaN,left_only


In [17]:
df_merge['ncessch'] = np.where(df_merge['School Name'] == 'Icahn Charter School 3', 
                               df_nces_raw[df_nces_raw['schnam'] == 'ICAHN CHARTER SCHOOL 3']['ncessch'],
                              df_merge['ncessch'])

df_merge['ncessch'] = np.where(df_merge['School Name'] == 'Bronx Academy of Promise Charter School', 
                               df_nces_raw[df_nces_raw['schnam'] == 'BRONX ACADEMY OF PROMISE CHARTER SCHOOL']['ncessch'],
                              df_merge['ncessch'])

In [18]:
df_merge.drop(['_merge', 'schnam', 'seasch', 'lstree', 'lcity', 'primary_address_line_1'], axis = 1, inplace = True)
df_merge.rename(columns = {'DBN': 'dbn', 'School Name': 'school_name', 'beds number': 'beds'}, inplace = True)

Now we have a crosswalk that matches up 3 different types of school IDs: 6-digit local NYC district borough numbers (`dbn`), 12-digit NY State's Basic Education Data System numbers (`beds`), and 12-digit federal NCES school IDs (`ncesssch`):

In [19]:
df_merge.head()

,dbn,school_name,beds,location_name,ncessch
0,01M015,P.S. 015 Roberto Clemente,310100010015,P.S. 015 Roberto Clemente,360007602087
1,01M019,P.S. 019 Asher Levy,310100010019,P.S. 019 Asher Levy,360007602101
2,01M020,P.S. 020 Anna Silver,310100010020,P.S. 020 Anna Silver,360007602104
3,01M034,P.S. 034 Franklin D. Roosevelt,310100010034,P.S. 034 Franklin D. Roosevelt,360007602157
4,01M063,The STAR Academy - P.S.63,310100010063,The STAR Academy - P.S.63,360007602270


## Federal (`NCESSCH`) to Federal (`COMBOKEY`)

The NCES is a separate branch of ED from the Office for Civil Rights (OCR), which produces the Civil Rights Data Collection survey that we are using. OCR identifies schools using **a 12-digit ID, aka the `combokey`.** In most cases, this is identical to the federal NCES school ID, the `NCESSCH`. However, in some cases--notably in NYC schools--it is not. For more detail on the differences between these two IDs for NYC schools, see Appendix D of ["Documentation to the NCES Common Core of Data Local Education Agency Universe Survey: School Year 2013–14"](https://nces.ed.gov/ccd/pdf/2015147_2013-14_LEA_documentation_v1a.pdf) (published July 2015).

Let's take a look at one of the intermediate files we created in the `federal_civil_rights_survey.ipynb` notebook to see this in action:

In [20]:
df_ocr = pd.read_feather(
    os.path.join(intermediate_dir, 'ocr_schools_ids.feather')
)

df_ocr.iloc[58276:58281]

,sch_name,combokey,nces_school_id
58276,PS 51 BRONX NEW SCHOOL,362058000127,360008700127
58277,PS 23 THE NEW CHILDREN'S SCHOOL,362058000130,360008700130
58278,IS 218 SALOME URENA,362058000152,360008300152
58279,PS 178 DR SELMAN WAKSMAN,362058000158,360008800158
58280,PS 279 CAPT MANUEL RIVERA JR,362058000183,360008700183


Not all of these are NYC schools (though we have shown five NYC schools above), since OCR conducts a nationwide survey. Also, not all schools that have `combokey` values have `nces_school_id` values. This is because of differences in the way that OCR and NCES define the universe of schools. For more information on the different survey populations for federal school statistics, see the OCR's ["Public-Use Data File User's Manual"](https://ocrdata.ed.gov/Downloads/CRDC%202013-14%20Public%20Use%20Data%20File%20Users%20Manual.pdf)  (published March 2016).

Let's merge the federal dataset with the schools from the NYC School Survey universe using the `nces_school_id` and the `ncessch`.

In [21]:
df_merge_ocr = pd.merge(df_merge, 
                        df_ocr, 
                        left_on = 'ncessch', 
                        right_on = 'nces_school_id', 
                        how = 'left', 
                        indicator = True)

There are only 10 schools in the NYC School Survey data that don't match with the federal OCR data. Doing a quick search of these school names on the [OCR school search tool](https://ocrdata.ed.gov/DistrictSchoolSearch#schoolSearch) shows that none of these unmatched schools are in the Office for Civil Rights database. They do not have federal statistics on bullying and harassment and so will be excluded from our analysis.

In [22]:
df_merge_ocr._merge.value_counts()

both          1752
left_only       10
right_only       0
Name: _merge, dtype: int64

In [23]:
df_merge_ocr[df_merge_ocr['_merge'] == 'left_only']['school_name'].values

array(['Research and Service High School',
       'Teaching Firms of America-Professional Preparatory Charter School',
       'The Ethical Community Charter School (TECCS)',
       'New American Academy Charter School',
       'Innovate Manhattan Charter School',
       'Growing Up Green Charter School',
       'Merrick Academy - Queens Public Charter School',
       'International Leadership Charter High School',
       'South Bronx Classical Charter School II',
       'Bronx Global Learning Institute for Girls Charter School'], dtype=object)

In [24]:
df_merge_ocr = df_merge_ocr.loc[df_merge_ocr['_merge'] == 'both']

Multiple schools in the OCR data can have the same `ncessch` IDs (and different `combokey` ids). These appear to be affiliated charter schools that might sometimes consolidate their reporting (e.g., Achievement First Crown Heights Charter School vs. Achievement First Crown Heights Middle School). 

In [25]:
df_merge_ocr.loc[df_merge_ocr.duplicated(['ncessch'], keep = False)].head()

,dbn,school_name,beds,location_name,ncessch,sch_name,combokey,nces_school_id,_merge
1577,84K356,Achievement First Crown Heights Charter School,331700860879,Achievement First Crown Heights Charter School,360014705821,ACHIEVEMENT FIRST CROWN HEIGHTS CHARTER SCHOOL,360014705821,360014705821,both
1578,84K356,Achievement First Crown Heights Charter School,331700860879,Achievement First Crown Heights Charter School,360014705821,ACHIEVEMENT FIRST CROWN HEIGHTS MIDDLE SCHOOL,360014799999,360014705821,both
1580,84K358,Achievement First East New York Charter School,331900860880,Achievement First East New York Charter School,360014905824,ACHIEVEMENT FIRST EAST NEW YORK CHARTER SCHOOL,360014905824,360014905824,both
1581,84K358,Achievement First East New York Charter School,331900860880,Achievement First East New York Charter School,360014905824,ACHIEVEMENT FIRST EAST NEW YORK MIDDLE SCHOOL,360014999999,360014905824,both
1593,84K508,Achievement First Endeavor Charter School,331300860902,Achievement First Endeavor Charter School,360017005867,ACHIEVEMENT FIRST ENDEAVOR CHARTER SCHOOL,360017005867,360017005867,both


Let's drop these schools, since we can't be not sure how to deal with their responses when comparing to the NYC School Survey responses.

In [26]:
df_merge_ocr = df_merge_ocr.loc[~df_merge_ocr.duplicated(['ncessch'])]
df_merge_ocr.drop(['sch_name', 'nces_school_id', '_merge'], axis = 1, inplace = True)

## The crosswalk

Now we have a crosswalk that matches up 4 different types of school IDs: 6-digit local NYC district borough numbers (`dbn`), 12-digit NY State's Basic Education Data System numbers (`beds`), 12-digit federal NCES school IDs (`ncessch`), and 12-digit federal OCR school IDs (`combokey`).

In [27]:
df_merge_ocr.head()

,dbn,school_name,beds,location_name,ncessch,combokey
0,01M015,P.S. 015 Roberto Clemente,310100010015,P.S. 015 Roberto Clemente,360007602087,362058002087
1,01M019,P.S. 019 Asher Levy,310100010019,P.S. 019 Asher Levy,360007602101,362058002101
2,01M020,P.S. 020 Anna Silver,310100010020,P.S. 020 Anna Silver,360007602104,362058002104
3,01M034,P.S. 034 Franklin D. Roosevelt,310100010034,P.S. 034 Franklin D. Roosevelt,360007602157,362058002157
4,01M063,The STAR Academy - P.S.63,310100010063,The STAR Academy - P.S.63,360007602270,362058002270


Ultimately, all but 40 schools from the NYC School Survey are included in our crosswalk.

In [28]:
len(df_nyc) - len(df_merge_ocr)

40

In [29]:
df_nyc.loc[~df_nyc['School Name'].isin(df_merge_ocr['school_name'])]['School Name'].values

array(['Washington Irving Yabc', 'George Washington Yabc',
       'Alfred E. Smith Campus YABC', 'Herbert H. Lehman Yabc',
       'Stevenson Yabc', 'Walton Yabc', 'Kennedy Yabc', 'Grace Dodge Yabc',
       'C Columbus Hs Yabc', 'Learning To Work Yabc At Monroe Academy',
       'Downtown Brooklyn Yabc', 'Automotive High School Yabc',
       "P.S. 372 -The Children's School",
       'Research and Service High School', 'BOYS AND GIRLS YABC',
       'Erasmus Yabc', 'South Shore Educational Complex Yabc',
       'Thomas Jefferson Yabc', 'Franklin K. Lane Campus YABC',
       'Franklin Delano Roosevelt YABC', 'Abraham Lincoln Yabc',
       'HS Arts & Business Yabc', 'Flushing Yabc', 'John Adams Yabc',
       'Staten Island Yabc',
       'Teaching Firms of America-Professional Preparatory Charter School',
       'The Ethical Community Charter School (TECCS)',
       'New American Academy Charter School',
       'Achievement First High School', 'Uncommmon Charter HS 2',
       'Achievement Fir

Since this crosswalk (the `df_merge_ocr` dataframe) might be useful for future research, let's save it to a csv file in the `data/output` folder.

In [30]:
df_merge_ocr.to_csv(os.path.join(output_dir, 'crosswalk.csv'), index = False)

## Merge NYC School Survey with the Federal Civil Rights survey using the crosswalk

Read in the processed federal OCR and NYC School Survey files

In [31]:
ocr = pd.read_feather(
    os.path.join(intermediate_dir, 'federal_ocr_survey.feather')
)
nyc_survey = pd.read_feather(
    os.path.join(intermediate_dir, 'nyc_survey_wide.feather')
)

Use the crosswalk, `df_merge_ocr`, to get the `dbn` for each of the NYC schools

In [32]:
ocr = pd.merge(ocr, 
               df_merge_ocr[['dbn', 'combokey']], 
               how = 'right', 
               on = 'combokey')

Match up the federal OCR survey with the NYC School Survey using this `dbn`

In [33]:
data = pd.merge(nyc_survey, 
                ocr, 
                on = 'dbn', 
                how = 'outer', 
                indicator = True)

As stated in the previous section, 40 school do not match

In [34]:
data[data['_merge'] == 'left_only']['school_name'].nunique()

40

In [35]:
data = data.loc[data['_merge'] == 'both']
data.drop(['_merge'], axis = 1, inplace = True)
data.reset_index(inplace = True, drop = True)

In [36]:
data.head()

,dbn,school_name,answer_code,dk_parents_perc_harass_parents,dk_parents_perc_harass_differences_parents,perc_harass_parents,perc_harass_students,perc_harass_teachers,perc_harass_differences_parents,perc_harass_differences_students,...,allegations_harass_dis_ind,students_report_harass_race_tot_ind,students_disc_harass_race_tot_ind,allegations_harass_race_ind,students_report_harass_sex_tot_ind,students_disc_harass_sex_tot_ind,allegations_harass_sex_ind,disc_harass_ind,report_harass_ind,allegations_harass_ind
0,01M015,P.S. 015 Roberto Clemente,1,20.000000,26.562500,54.166667,NaN,36.363636,63.829787,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01M015,P.S. 015 Roberto Clemente,2,20.000000,26.562500,16.666667,NaN,54.545455,17.021277,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01M015,P.S. 015 Roberto Clemente,3,20.000000,26.562500,14.583333,NaN,9.090909,6.382979,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01M015,P.S. 015 Roberto Clemente,4,20.000000,26.562500,14.583333,NaN,0.000000,12.765957,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01M019,P.S. 019 Asher Levy,1,19.917012,22.362869,35.751295,NaN,54.054054,41.304348,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Save combined data

Save the `ocr` dataframe, which represents the cleaned Civil Rights Data Collection survey data, to a [feather](https://blog.cloudera.com/blog/2016/03/feather-a-fast-on-disk-format-for-data-frames-for-r-and-python-powered-by-apache-arrow/) file in the `data/intermediate` folder.

In [37]:
data.to_feather(os.path.join(intermediate_dir, 'combined_surveys.feather'))